In [ ]:
from rotten_tomatoes_client import RottenTomatoesClient

In [1]:
import requests

In [2]:
import re
import numpy as np
import pandas as pd
import csv
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
movie_meta = pd.read_csv('app/irsystem/models/MovieSummaries/movie.metadata.tsv', header=None, delimiter='\t',encoding='utf-8',names=['WikiID', 'FreebaseID', 'Title', 'ReleaseDate', 'Revenue', 'Runtime', 'Languages', 'Countries', 'Genres'])
movie_meta = movie_meta.drop(['FreebaseID','Revenue'], axis=1)
movie_meta.head()

,WikiID,Title,ReleaseDate,Runtime,Languages,Countries,Genres
0,975900,Ghosts of Mars,2001-08-24,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,Brun bitter,1988,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,White Of The Eye,1987,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,A Woman in Flames,1983,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [4]:
plots = pd.read_csv('app/irsystem/models/MovieSummaries/plot_summaries.txt', sep='\t',names=['WikiID', 'Summary'])
plots.head()

,WikiID,Summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [5]:
merged = pd.merge(movie_meta, plots, on='WikiID')
print(len(merged))
merged.head()

42204


,WikiID,Title,ReleaseDate,Runtime,Languages,Countries,Genres,Summary
0,975900,Ghosts of Mars,2001-08-24,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...","Set in the second half of the 22nd century, th..."
1,9363483,White Of The Eye,1987,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",A series of murders of rich young women throug...
2,261236,A Woman in Flames,1983,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}","Eva, an upper class housewife, becomes frustra..."
3,18998739,The Sorcerer's Apprentice,2002,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0hzlz"": ""South Africa""}","{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...","Every hundred years, the evil Morgana returns..."
4,6631279,Little city,1997-04-04,93.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ...","Adam, a San Francisco-based artist who works a..."


In [6]:
analyzer = SentimentIntensityAnalyzer()

In [7]:
def get_sentiment(df):
    comp = []
    pos = []
    neg = []
    neu = []
    for x in df['Summary']:
        sentiment = analyzer.polarity_scores(x)
        comp.append(sentiment['compound'])
        pos.append(sentiment['pos'])
        neg.append(sentiment['neg'])
        neu.append(sentiment['neu'])
    df['compound'] = comp
    df['pos'] = pos
    df['neg'] = neg
    df['neu'] = neu

In [8]:
genres = []
for x in merged['Genres']:
    g = re.findall(': \"(.*?)\"', x)
    genres.append(g)
merged['Genres'] = genres

In [ ]:
get_sentiment(merged)

In [ ]:
merged.head()

In [ ]:
#merged.to_csv (r'merged_data.csv', index = True, header=True)

In [9]:
omdb_TOKEN = '4292bf53'

In [ ]:
def ResizeList(l, size, fill_with=None):
    l = l + [fill_with]*(size-len(l))

In [11]:
def response(result):
    text = result[result.find('"Response":')+12:]
    return text.find('True') > -1

In [12]:
def cleanjson(result):
    title = result[result.find("Title")+8:result.find("Year")-3]
    plot = result[result.find("Plot")+7:result.find("Language")-3]
    try:
        review_imdb = float(result[result.find(
        '"Internet Movie Database","Value":"')+35:result.find('Source":"Rotten Tomatoes"')-8])
    except:
        review_imdb = -1
    try:
        review_rotten = float(result[result.find(
        'Source":"Rotten Tomatoes","Value":')+35: result.find('},{"Source":"Metacritic"')-2])
    except:
        review_rotten = -1
    return [title, plot, review_imdb, review_rotten]

In [15]:
def get_ratings(df):
    imdb_rat = []
    rotten_rat = []
    i = 0
    while (i < 42204):
    #for i in range(0, 42204):
        #print(i)
        title = df['Title'][i]
        query = "http://www.omdbapi.com/?apikey=" + omdb_TOKEN + "&t=" + title
        params = {"r": "json", "plot": "full"}
        result = requests.get(query, params)
        #if response(result.text):
        json = cleanjson(result.text)
        review_imdb = json[2]
        imdb_rat.append(review_imdb)
        review_rotten = json[3]
        rotten_rat.append(review_rotten)
        #else:
            #imdb_rat.append(None)
            #rotten_rat.append(None)
        i = i + 1
            
    df['IMDB Ratings'] = imdb_rat
    df['Rotten Tomatoes score'] = rotten_rat
    
        #result = RottenTomatoesClient.search(term = x)
        #print(result['movies']['meterScore'])
        

In [16]:
#Takes about forty minutes to run
get_ratings(merged)

In [17]:
merged.head()

,WikiID,Title,ReleaseDate,Runtime,Languages,Countries,Genres,Summary,IMDB Ratings,Rotten Tomatoes score
0,975900,Ghosts of Mars,2001-08-24,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","[Thriller, Science Fiction, Horror, Adventure,...","Set in the second half of the 22nd century, th...",4.9,22.0
1,9363483,White Of The Eye,1987,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","[Thriller, Erotic thriller, Psychological thri...",A series of murders of rich young women throug...,6.2,-1.0
2,261236,A Woman in Flames,1983,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}",[Drama],"Eva, an upper class housewife, becomes frustra...",-1.0,-1.0
3,18998739,The Sorcerer's Apprentice,2002,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0hzlz"": ""South Africa""}","[Family Film, Fantasy, Adventure, World cinema]","Every hundred years, the evil Morgana returns...",6.1,40.0
4,6631279,Little city,1997-04-04,93.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","[Romantic comedy, Ensemble Film, Comedy-drama,...","Adam, a San Francisco-based artist who works a...",-1.0,-1.0


In [ ]:
# if the rating is -1 then no rating available
# looking for better API that might give all movies a rating
# will move this to movie_data_processing by Sunday night